In [1]:
import numpy as np
import tensorflow as tf
import keras as k
import random
import sys

Using TensorFlow backend.


In [2]:
data = open('names.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique ones.' % (data_size, vocab_size))

There are 6941 total characters and 27 unique ones.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
maxlen = 10
step = 3

sentences = []
next_chars = []
for i in range(0, len(data) - maxlen, step):
    sentences.append(data[i: i + maxlen])
    next_chars.append(data[i + maxlen])
print('Number of sequences:', len(sentences))
print('First 10 sequences and next chars:')
for i in range(10):
    print('[{}]:[{}]'.format(sentences[i], next_chars[i]))

Number of sequences: 2311
First 10 sequences and next chars:
[mary
patri]:[c]
[y
patricia]:[
]
[atricia
li]:[n]
[icia
linda]:[
]
[a
linda
ba]:[r]
[inda
barba]:[r]
[a
barbara
]:[e]
[arbara
eli]:[z]
[ara
elizab]:[e]
[
elizabeth]:[
]


In [5]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_ix[char]] = 1
    y[i, char_to_ix[next_chars[i]]] = 1
print('Size of X: {:.2f} MB'.format(X.nbytes/1024/1024))
print('Size of y: {:.2f} MB'.format(y.nbytes/1024/1024))

Vectorization...
Size of X: 0.60 MB
Size of y: 0.06 MB


In [6]:
nb_units = 64

model = k.Sequential()

model.add(k.layers.LSTM(nb_units, input_shape=(maxlen, len(chars))))

model.add(k.layers.Dense(units=len(chars)))
model.add(k.layers.Activation('softmax'))

optimizer = k.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                23552     
_________________________________________________________________
dense_1 (Dense)              (None, 27)                1755      
_________________________________________________________________
activation_1 (Activation)    (None, 27)                0         
Total params: 25,307
Trainable params: 25,307
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
class SampleResult(k.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):

        start_index = random.randint(0, len(data) - maxlen - 1)

        for diversity in [0.2, 0.5, 1.0, 1.2]:
            generated = ''
            sentence = data[start_index: start_index + maxlen]
            generated += sentence
            print()
            print('----- Generating with diversity',
                  diversity, 'seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(100):
                x = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x[0, t, char_to_ix[char]] = 1.

                preds = self.model.predict(x, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = ix_to_char[next_index]

                generated += next_char
                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
        print('\n\n')
sample_callback = SampleResult()

In [11]:
history = model.fit(X, y, 
                        epochs=10, 
                        batch_size=512,
                        verbose=2,
                       callbacks=[sample_callback])

Epoch 1/10
 - 0s - loss: 2.7483

----- Generating with diversity 0.2 seed: "lie
claude"
lie
claude
e



e
iee
eee

ee
e

ne
ta

eea

a
ee
e
e
e
e



ae
eee

ee

l
ee
e
eee
e

e
aee

te
e

a
re
e

e

----- Generating with diversity 0.5 seed: "lie
claude"
lie
claude

ie


saeiet
eidlenabn
nenyn
ianncae



aen
eet


iee
a

rnenaoth
eai
e
ia

yeameaniinencl
ai
m
e


----- Generating with diversity 1.0 seed: "lie
claude"
lie
claudecrarhk
lekeia
mcraneelsbclorlerey


iel
etciayavbsieayreelar
tyknratbsannelvnir
u
ia
ntrtg
a
nieea
t
----- Generating with diversity 1.2 seed: "lie
claude"
lie
claudeicn
qemeseriraloeieyskuaeneayecaeansenih


nrr
aatal
nellnetvlzenat

td
sp
ie

tna
lageiytidet
hioil


Epoch 2/10
 - 0s - loss: 2.7280

----- Generating with diversity 0.2 seed: "
keri
ursu"

keri
ursu
a
a
ae
e
a
a
ea
e

a
ae
e
e
are

eaa
a
eee
re
ee
a

eee
a
ee

aa
e
e
e
ea
aa
a
ea

a
ean
ee
ea

aar
----- Generating with diversity 0.5 seed: "
keri
ursu"

keri
ursuler
a

ae
ea
ce
leey

emerraraera
san